This code explores the relationship between the state WSB data and the SDWIS data.

Updated 3/18/22

Make a dataframe comparing percentages of pwsid matching between WSB and SDWIS data and displaying pwsid duplicate counts for states with staged WSB data.

In [32]:
import geopandas as gpd
import pandas as pd
import os
from dotenv import load_dotenv

In [33]:
# File path and data import
load_dotenv()

staging_path = os.environ["WSB_STAGING_PATH"]

In [34]:
# Helper: Divides and returns a percent

def get_pc(num, denom):
    return round((num/denom)*100, 1)

In [35]:
# get list of paths/filenames for staged state wsb data
staging_geojson_list = []
for dirpath, dirnames, filenames in os.walk(staging_path):
    for filename in [f for f in filenames if f.endswith("_wsb_labeled.geojson")]:
        staging_geojson_list.append(os.path.join(filename[:2], filename))

nested_list = []

In [36]:
# read in sdwis data
sdwis = pd.read_csv(os.path.join(staging_path, 'sdwis_water_system.csv'))

/var/folders/4w/zhsn7cr924bdcsncw_23jrf80000gn/T/ipykernel_48978/2437291838.py:2: DtypeWarning: Columns (32,34,38) have mixed types. Specify dtype option on import or set low_memory=False.
  sdwis = pd.read_csv(os.path.join(staging_path, 'sdwis_water_system.csv'))


In [37]:
print(f'Comparing WSB and SDWIS data for {len(staging_geojson_list)} states...\n')

# compare wsb staging data with sdwis
for staging_file in staging_geojson_list:
        
    # read in staged state wsb data
    # select state from sdwis data
    state_wsb = gpd.read_file(os.path.join(staging_path, staging_file))
    state = staging_file[:2].upper()
    state_sdwis = sdwis[sdwis['primacy_agency_code'] == state]
        
    # df id columns
    id_wsb = state_wsb['pwsid']
    id_sdwis = state_sdwis['pwsid']

    # df lengths
    len_wsb = len(state_wsb)
    len_sdwis = len(state_sdwis)

    # wsb id % matching to sdwis id
    wsb_matching_to_sdwis = len(state_wsb[state_wsb['pwsid'].isin(id_sdwis)])

    # sdwis id % matching to wsb id
    sdwis_matching_to_wsb = len(state_sdwis[state_sdwis['pwsid'].isin(id_wsb)])
        
    nested_list.append([state,
                        get_pc(wsb_matching_to_sdwis, len_wsb),
                        get_pc(sdwis_matching_to_wsb, len_sdwis),
                        get_pc(len_wsb, len_sdwis),
                        len(id_wsb) - len(set(id_wsb)),
                        len(id_sdwis) - len(set(id_sdwis))])
        
wsb_sdwis_matches = pd.DataFrame(nested_list, 
                                 columns=['state', 
                                          '% WSB IDs matching to SDWIS IDs',
                                          '% SDWIS IDs matching to WSB IDs',
                                          'WSB is % size of SDWIS', 
                                          'WSB dup IDs', 'SDWIS dup IDs'])

Comparing WSB and SDWIS data for 12 states...



In [38]:
wsb_sdwis_matches

,state,% WSB IDs matching to SDWIS IDs,% SDWIS IDs matching to WSB IDs,WSB is % size of SDWIS,WSB dup IDs,SDWIS dup IDs
0,MO,100.0,4.1,4.2,1,0
1,OK,99.9,12.7,12.7,0,0
2,CA,99.3,26.6,26.8,6,0
3,CT,99.4,4.6,4.8,11,0
4,TX,100.0,28.8,28.8,0,0
5,PA,99.9,8.0,8.0,3,0
6,NC,97.4,2.3,2.3,0,0
7,NM,100.0,22.6,23.0,11,0
8,NJ,100.0,4.0,4.0,2,0
9,KS,99.6,46.9,47.5,9,0
